In [ ]:
include("../src/graph.jl")
include("../src/io.jl")
include("../src/pr.jl")

In [ ]:
@info("loading graph")

core = SimpleDiGraph(UInt32)

load_mgs3_graph(core, "../datasets/Arxiv_HEP-PH/Arxiv_HEP-PH_core.mgs")
#load_mgs4_graph(core, "../datasets/Arxiv_HEP-PH/Arxiv_HEP-PH_core.mgz")

# define constants
n = nv(core)
damping = .85
epsilon = 1e-8

# source and target node to be used in the graph
s = convert(UInt32,1)
t = convert(UInt32,100)

In [ ]:
@info("getting rcore")
rcore = get_reverse_graph(core)

In [ ]:
@info("computing Pagerank of core and rcore")

@time pr_core = PR(core, rcore, epsilon=epsilon)
#@time pr_rcore = PR(rcore, core, epsilon=epsilon)

In [ ]:
@info("computing personalized Pageranks for nodes 1 and 100")
@time pr_s = PPR(s, core, rcore, epsilon=epsilon)
#@time pr_t = PPR(t, rcore, core, epsilon=epsilon)

In [ ]:
# compute Monte Carlo PR
niter = 100
@info("compute Pagerank (Monte Carlo)")
@time pr_mc = PR(core, niter)

@info("pr_core <-> pr_mc: ", chebyshev(pr_core, pr_mc))

In [ ]:
# matrix computation
@info("getting P matrix")
@time P = get_sparse_P_matrix(core)

In [ ]:
@info("compute Pagerank (power iteration)")
ppr = zeros(Float64,n)
ppr[s] = 1.
@time pr_pi = PR(P, epsilon=epsilon)

@info("computing personalized Pageranks for node 1 (power iteration)")
@time pr_pi_s = PR(P, ppr=ppr, epsilon=epsilon)

@info("pr_core <-> pr_pi: ", chebyshev(pr_core, pr_pi))
@info("pr_s <-> pr_pi_s: ", chebyshev(pr_s, pr_pi_s))

In [ ]:
@info("compute non-linear Pagerank (power iteration)")
ppr = zeros(Float64, n)
ppr[s] = 1.
# change damping
damping = 0.95
max_iter = 15

@time pr_pi_nl = PR(P, tanh, damping=damping, epsilon=epsilon, max_iter=max_iter)

@info("pr_pi <-> pr_pi_nl: ", chebyshev(pr_pi, pr_pi_nl))


In [ ]:
using GraphPlot

layout=(args...)->spring_layout(args...; C=20)
gplot(core, layout=layout)
